In [2]:
import os
import torch
import ffmpeg
import numpy as np
from transformers import (
    CLIPProcessor, 
    CLIPModel,
    pipeline
)
from moviepy.editor import *
import subprocess
from typing import List, Dict, Tuple, Set
import logging
import json
import cv2
import librosa
import whisper
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

c:\Users\aahan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def split_video_into_parts(video_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get the total duration of the video file
    cmd_duration = [
        'ffmpeg', '-i', video_path, 
        '-f', 'null', '/dev/null'
    ]
    
    result = subprocess.run(cmd_duration, capture_output=True, text=True)

    # Parse the duration from ffmpeg's output
    duration_line = [line for line in result.stderr.split('\n') if 'Duration' in line]
    if not duration_line:
        print("Could not find duration in the video file.")
        return

    # Extract duration in seconds
    duration = duration_line[0].split(' ')[3][:-1]  # Format is "00:00:00.00"
    hours, minutes, seconds = map(float, duration.split(':'))
    total_duration = hours * 3600 + minutes * 60 + seconds

    # Calculate the duration of each split part
    part_duration = total_duration / 5

    # Split the video into 5 parts
    for i in range(5):
        start_time = i * part_duration
        output_file = os.path.join(output_dir, f'part_{i+1}.mp4')  # Change extension if needed
        
        cmd_split = [
            'ffmpeg', '-y',  # Overwrite output files if they exist
            '-ss', str(start_time),  # Start time
            '-i', video_path,  # Input video file
            '-t', str(part_duration),  # Duration of each part
            '-c:v', 'copy',  # Copy the video stream without re-encoding
            '-c:a', 'copy',  # Copy the audio stream without re-encoding
            output_file
        ]
        
        # Run the command
        subprocess.run(cmd_split)

In [12]:
split_video_into_parts(video_path='videoplayback(1).mp4',output_dir='./random/')